In [1]:
%run ../Helpers

In [2]:
impressions = spark.read.parquet('dbfs:/mnt/nycdsa/quentin/impressions_final2/') # 4,714,433 count, training where day < 12
# impressions = spark.read.parquet('dbfs:/mnt/nycdsa/quentin/impressions_20ads/')

In [3]:
impressions = cleanup_gender(impressions)

In [4]:
age_as_category=1
if(age_as_category):
  impressions = cleanup_age_category(impressions)
else:
  impressions = cleanup_age_numeric(impressions) 

In [5]:
impressions = cleanup_timestamp(impressions)

In [6]:
impressions = format_region(impressions)
impressions = reduce_cardinality(impressions, "region", 55, mode=0)

In [7]:
impressions = iab_encoder(impressions, "iabCategories")

In [8]:
impressions = clean_landingPage(impressions)

In [9]:
impressions = clean_bestVenueName(impressions)

In [10]:
# from pyspark.sql.functions import col

# impressions = interaction(impressions, col("campaign"), col("bestVenueName"), "campaign-bestVenueName")
# impressions = interaction(impressions, col("landingPage"), col("bestVenueName"), "landingPage-bestVenueName")

In [11]:
print("Categorical variables in impressions dataframe: ")
print([item[0] for item in impressions.dtypes if item[1].find('string') > -1])
print("\r")
categoricalColumns = ['adSize', 'adType', 'ageGroup', 'bestVenueName', 'carrier', 'deviceName', 'deviceType', 'exchange', 'gender', 'landingPage', 'os', 'region', 'timestamp_hour', 'timestamp_weekday', 'targetGroup', 'zip'] 

# targetGroup numeric treated as categorical
print("Categorical variables used in model: ")
print(categoricalColumns)

In [12]:
all_numerical_columns = [item[0] for item in impressions.dtypes if (item[1].find('int') > -1) | (item[1].find('double') > -1)] 
print("Numerical variables in impressions dataframe: ")
print(all_numerical_columns)
print("\r")
excluded_numerical = ['backendStatus', 'bidPrice', 'elbStatus', 'location', 'month', 'price', '', 'year', 'TrainTestFlag']
print("Excluded numerical variables: ")
print(excluded_numerical)
print("\r")
numericalColumns = [x for x in all_numerical_columns if x not in excluded_numerical]
print("Numerical variables used in model (including computed): ")
print(numericalColumns)

In [13]:
impressions = impressions.select(*(categoricalColumns + numericalColumns))

In [14]:
display(impressions)

In [15]:
impressions.printSchema()

In [16]:
# impressions.write.mode('overwrite').parquet('dbfs:/mnt/nycdsa/quentin/impressions_pre_processed/')

In [17]:
display(dbutils.fs.ls('dbfs:/mnt/nycdsa/quentin/'))

In [18]:
impressions = spark.read.parquet('dbfs:/mnt/nycdsa/quentin/impressions_pre_processed/') # 4714433 count, training where day < 12

In [19]:
train, test = impressions.filter(impressions.day<12),\
              impressions.filter(impressions.day >= 12) # predict next days
train.cache()
test.cache()

In [20]:
train.groupby('clicked').count().show()
# print(test.count())

In [21]:
# train.groupBy("day").agg({"*": "count"}).show()

In [22]:
from datetime import datetime
from math import exp, log, sqrt
import mmh3
import sys
reload(sys)
sys.setdefaultencoding('utf8') # prevents ASCII crash
# import pyspark class Row from module sql
from pyspark.sql import Row


##############################################################################
# class, function, generator definitions #####################################
##############################################################################

class ftrl_proximal(object):
    ''' Follow the regularized leader - proximal
        this is an adaptive-learning-rate sparse logistic-regression with
        efficient L1-L2-regularization
    '''

    def __init__(self, alpha, beta, L1, L2, D, interaction, epoch):
        # parameters
        self.alpha = alpha
        self.beta = beta
        self.L1 = L1
        self.L2 = L2
        self.epoch = epoch

        # feature related parameters
        self.D = D
        self.interaction = interaction

        # model
        # n: squared sum of past gradients
        # z: weights
        # w: lazy weights
        self.n = [0.] * D
        self.z = [0.] * D
        self.w = {}

    def _indices(self, x):
        ''' A helper generator that yields the indices in x

            The purpose of this generator is to make the following
            code a bit cleaner when doing feature interaction.
        '''

        # first yield index of the bias term
        yield 0

        # then yield the normal indices
        for index in x:
            yield index

        # now yield interactions (if applicable)
        if self.interaction:
            D = self.D
            L = len(x)

            x = sorted(x)
            for i in xrange(L):
                for j in xrange(i+1, L):
                    # one-hot encode interactions with hash trick
                    yield mmh3.hash(str(x[i]) + '_' + str(x[j]), signed=False) % D

    def get_prob(self, x):
        ''' Get probability estimation on x

            INPUT:
                x: features

            OUTPUT:
                probability of p(y = 1 | x; w)
        '''

        # parameters
        alpha = self.alpha
        beta = self.beta
        L1 = self.L1
        L2 = self.L2

        # model
        n = self.n
        z = self.z
        w = {}

        # wTx is the inner product of w and x
        wTx = 0.
        for i in self._indices(x):
            sign = -1. if z[i] < 0 else 1.  # get sign of z[i]

            # build w on the fly using z and n, hence the name - lazy weights
            # we are doing this at prediction instead of update time is because
            # this allows us for not storing the complete w
            if sign * z[i] <= L1:
                # w[i] vanishes due to L1 regularization
                w[i] = 0.
            else:
                # apply prediction time L1, L2 regularization to z and get w
                w[i] = (sign * L1 - z[i]) / ((beta + sqrt(n[i])) / alpha + L2)

            wTx += w[i]

        # cache the current w for update stage
        self.w = w

        # bounded sigmoid function, this is the probability estimation
        return 1. / (1. + exp(-max(min(wTx, 35.), -35.)))

    def update(self, x, p, y):
        ''' Update model using x, p, y

            INPUT:
                x: feature, a list of indices
                p: click probability prediction of our model
                y: answer

            MODIFIES:
                self.n: increase by squared gradient
                self.z: weights
        '''

        # parameter
        alpha = self.alpha

        # model
        n = self.n
        z = self.z
        w = self.w

        # gradient under logloss
        g = p - y

        # update z and n
        for i in self._indices(x):
            sigma = (sqrt(n[i] + g * g) - sqrt(n[i])) / alpha
            z[i] += g - sigma * w[i]
            n[i] += g * g
            
            
    def fit(self, df): 
      start = datetime.now()
      
      D = self.D
      epoch = self.epoch

      # start training
      for e in xrange(epoch):
          loss = 0.
          count = 0

          for t, x, y in data(df, D):  # data is a generator
              #    t: just a instance counter
              #    x: features
              #    y: label (click)

              # step 1, get prediction from learner
              p = learner.get_prob(x)

              if (holdout and t % holdout == 0):
                  # step 2-1, calculate validation loss
                  #           we do not train with the validation data so that our
                  #           validation loss is an accurate estimation
                  #
                  # holdout: validate with every N instance, train with others
                  loss += logloss(p, y)
                  count += 1
              else:
                  # step 2-2, update learner with label (click) information
                  self.update(x, p, y)

          if (holdout):
            print('Epoch %d finished, validation logloss: %f, elapsed time: %s' % (
              e, loss/count, str(datetime.now() - start)))
          
    def predict(self, df):  # add as method 'predict' of ftrl_proximal class
      
      D = self.D
      
      prob_list = list()
      for t, x, y in data(df, D):
        p = self.get_prob(x)
        prob_list.append(p)
    #     print(t, p)
      return prob_list
    
    def save(self, filename, folder_path="/mnt/nycdsa/quentin/models/"):
      import os
      
      n = self.n
      z = self.z

      model = sqlContext.createDataFrame([Row(n= n[i], z=z[i]) for i in range(len(n))])
      model_path = os.path.join(folder_path, filename)
      model.coalesce(1).write.csv(model_path, mode='overwrite')

In [23]:
def logloss(p, y):
    ''' FUNCTION: Bounded logloss

        INPUT:
            p: our prediction
            y: real answer

        OUTPUT:
            logarithmic loss of p given y
    '''

    p = max(min(p, 1. - 10e-15), 10e-15)
    return -log(p) if y == 1. else -log(1. - p)


def data(df, D):
    ''' GENERATOR: Apply hash-trick to input dataframe
                   and for simplicity, we one-hot-encode everything

        INPUT:
            path: path to training or testing file
            D: the max index that we can hash to

        YIELDS:
            x: a list of hashed and one-hot-encoded 'indices'
               we only need the index since all values are either 0 or 1
            y: y = 1 if we have a click, else we have y = 0
    '''
      
    for t, row in enumerate(df.rdd.toLocalIterator()):
        # process clicks
        y = 0.
        if 'clicked' in row:
            if row['clicked'] == 1:
              y = 1.
        
        # build x
        x = []
        for key in df.columns:
          if (key != 'clicked'): # do not use label!
            value = str(row[key]).encode('utf-8')
            # one-hot encode everything with hash trick
            index = hash(key + '_' + value) % D
            x.append(index)
        yield t, x, y

In [24]:
holdout = None
params = {'alpha': .1, # learning rate
              'beta': 1., # smoothing parameter for adaptive learning rate
             'L1': 0.05, # L1 regularization, larger value means more regularized
             'L2': 0.05, # L2 regularization, larger value means more regularized
             'D': 2 ** 20, # number of weights to use
             'interaction': False, # whether to enable poly2 feature interactions
             'epoch': 1} # learn training data for N passes

# initialize learner
learner = ftrl_proximal(**params)

learner.fit(train)

In [25]:
# distinct features 
print(len(learner.z) - learner.z.count(0))
learner.z[:10]

In [26]:
learner.save("model_zip_targetGroup", "/mnt/nycdsa/quentin/models/")

In [27]:
##############################################################################
# Prediction  ##########################
##############################################################################
test = test[test['day'].isin([12, 13, 14, 15, 16, 17, 18, 19])]
probas = learner.predict(test.drop("clicked"))
probas


In [28]:
import numpy as np
probas_array = np.asarray(probas)
clicks_array = np.asarray(test.select('clicked').collect())

In [29]:
model_FTRL = sqlContext.createDataFrame([Row(np.asscalar(clicks_array[i]), np.asscalar(probas_array[i])) for i in range(len(clicks_array))], 
                                       ["Labels", "Probability"])
model_FTRL.write.mode('overwrite').parquet('dbfs:/mnt/nycdsa/quentin/graph/model_FTRL_7_days_test')

In [30]:
from sklearn.metrics import roc_auc_score
from sklearn.metrics import average_precision_score
from sklearn.metrics import log_loss

print("roc_auc_score",
      roc_auc_score(clicks_array, probas_array, average="macro"))

print("average_precision_score",
      average_precision_score(clicks_array, probas_array, average="macro"))

print("log_loss: ",
      log_loss(clicks_array, probas_array, eps=1e-15, normalize=True))

In [31]:
# display(model_FTRL)

In [32]:
display(dbutils.fs.ls('/mnt/nycdsa/quentin/graph/'))

In [33]:
# Models to load
simple_logistic = spark.read.parquet('dbfs:/mnt/nycdsa/quentin/graph/simple_logistic/')

# model_dt = spark.read.parquet('dbfs:/mnt/nycdsa/quentin/graph/model_dt/')
model_dt3deep = spark.read.parquet('dbfs:/mnt/nycdsa/quentin/graph/model_dt3deep/').withColumnRenamed('clicked', 'Labels').withColumnRenamed('probability', 'Probability')
model_rf = spark.read.parquet('dbfs:/mnt/nycdsa/quentin/graph/model_rf/').withColumnRenamed('clicked', 'Labels').withColumnRenamed('probability', 'Probability')

model_FTRL = spark.read.parquet('dbfs:/mnt/nycdsa/quentin/graph/model_FTRL_7_days_test/')
FTRL_full_data = spark.read.parquet('dbfs:/mnt/nycdsa/quentin/graph/model_FTRL_full_data/') 

In [34]:
# display(simple_logistic)
# display(model_dt)
display(model_dt3deep)
# display(model_rf)
# display(FTRL_full_data)

In [35]:
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
import numpy as np

# Models:
# dbfs:/mnt/nycdsa/quentin/graph/model_FTRL
# dbfs:/mnt/nycdsa/quentin/graph/simple_logistic

models = [simple_logistic, model_dt3deep, model_rf, model_FTRL, FTRL_full_data]

labels, scores = [], []
fpr, tpr, auroc = [], [], []

# Read all models
for model in models:
  labels = np.array(model.select('Labels').collect())
  scores = np.array(model.select('Probability').collect())
  
  fpr_temp, tpr_temp, _ = roc_curve(labels, scores)
#   precision_recall_curve
  fpr.append(fpr_temp)
  tpr.append(tpr_temp)
  auroc.append(roc_auc_score(labels, scores, average="macro"))  

In [36]:
import matplotlib.pyplot as plt

model_names = ['Simple Logistic', '3-level Decision Tree', 'Random Forest', 'FTLR', 'FTRL_full_data']
fig, ax = plt.subplots()
lw = 2
for i in range(len(models)):
  ax.plot(fpr[i], tpr[i], lw=lw) 
ax.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
ax.set_xlim([0.0, 1.0])
ax.set_ylim([0.0, 1.05])

ax.set_xlabel('False Positive Rate')
ax.set_ylabel('True Positive Rate')
ax.set_title('Receiver operating characteristic curve')
ax.legend(['%s (area = %0.3f)' % (model_names[i], auroc[i]) for i in range(len(model_names))], loc="lower right")
# legend(loc="lower right")

display(fig)

In [37]:
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import average_precision_score
import numpy as np

# Models:
# dbfs:/mnt/nycdsa/quentin/graph/model_FTRL
# dbfs:/mnt/nycdsa/quentin/graph/simple_logistic

models = [simple_logistic, model_dt3deep, model_rf, model_FTRL, FTRL_full_data]

labels, scores = [], []
precision, recall, pr = [], [], []

# Read all models
for model in models:
  labels = np.array(model.select('Labels').collect())
  scores = np.array(model.select('Probability').collect())
  
  precision_temp, recall_temp, _ = precision_recall_curve(labels, scores) # don't need to save thresholds
#   precision_recall_curve
  precision.append(precision_temp)
  recall.append(recall_temp)
  pr.append(average_precision_score(labels, scores))  

In [38]:
# min(precision[0][1:]-precision[0][:-1])
simple_logistic.agg({'Probability':'min'}).show()

In [39]:
import matplotlib.pyplot as plt

model_names = ['Simple Logistic', '3-level Decision Tree', 'Random Forest', 'FTLR', 'FTRL_full_data']
fig, ax = plt.subplots()
lw = 2
for i in range(len(models)):
  ax.plot(precision[i], recall[i], lw=lw) 
ax.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
ax.set_xlim([0.0, 1.0])
ax.set_ylim([0.0, 1])

ax.set_xlabel('Recall')
ax.set_ylabel('Precision')
ax.set_title('Precision-Recall curve')
ax.legend(['%s (area = %0.3f)' % (model_names[i], pr[i]) for i in range(len(model_names))], loc="upper right")

display(fig)

In [40]:
# Full data AUPR
from sklearn.metrics import average_precision_score
print("average_precision_score",
      average_precision_score(np.array(FTRL_full_data_df.select('Labels').collect()),\
                              np.array(FTRL_full_data_df.select('Probability').collect()), average="macro"))

In [41]:
from sklearn.metrics import log_loss


models = [simple_logistic, model_dt3deep, model_rf, model_FTRL, FTRL_full_data]

labels, scores = [], []
ll = []
ll_temp = 0

# Read all models
for model in models:
  labels = np.array(model.select('Labels').collect())
  scores = np.array(model.select('Probability').collect())
  ll_temp = log_loss(labels, scores, eps=1e-15, normalize=True)
  ll.append(ll_temp)
  print("log_loss: ", ll_temp)

In [42]:
np.mean(labels) # 0.46% average CTR

In [43]:
labels = np.array(simple_logistic.select('Labels').collect())
scores = np.array([np.mean(labels)] * labels.shape[0])

print("AUROC", roc_auc_score(labels, scores, average="macro"))
print("AUPR", average_precision_score(labels, scores))
print("LogLoss: ", log_loss(labels, scores, eps=1e-15, normalize=True))

In [44]:
FTRL_full_data = spark.read.parquet('dbfs:/mnt/nycdsa/quentin/graph/model_FTRL_full_data/')

In [45]:
thresholds = [0.005, 0.01, 0.05, 0.1, 0.15, 0.2, 0.5]

for t in thresholds:
  print("threshold : ", t)
  FTRL_full_data.withColumn("Prediction", FTRL_full_data["Probability"] >= t).crosstab("Prediction", "Labels").show()

In [46]:
from sklearn.model_selection import ParameterGrid
import numpy as np
from sklearn.metrics import roc_auc_score
from sklearn.metrics import average_precision_score

# holdout for logloss calculation
holdout= None # use every N training instance for holdout validation
# labels
clicks_array = np.asarray(test.select('clicked').collect())

param_grid = {'alpha': [.1, .05], # learning rate
              'beta': [1., 0.5], # smoothing parameter for adaptive learning rate
             'L1': [0., .5, 1.], # L1 regularization, larger value means more regularized
             'L2': [0., .5, 1.], # L2 regularization, larger value means more regularized
             'D': [2 ** 20], # number of weights to use
             'interaction': [False], # whether to enable poly2 feature interactions
             'epoch': [1]} # learn training data for N passes


grid = ParameterGrid(param_grid)

for params in grid:
  # print parameters
  print(params)
  
  # train
  learner = ftrl_proximal(**params)
  learner.fit(train)
  print("Non-zero weights: ", len(learner.z) - learner.z.count(0))
  
  # save model
  learner.save("model_created_on_" + str(datetime.now()), "/mnt/nycdsa/quentin/models/")
  
  # predict
  probas_array = np.asarray(learner.predict(test.drop("clicked")))
  print("roc_auc_score",
      roc_auc_score(clicks_array, probas_array, average="macro"))
  print("average_precision_score",
      average_precision_score(clicks_array, probas_array, average="macro"))

In [47]:
display(dbutils.fs.ls("/mnt/nycdsa/quentin/models"))

In [48]:
# parameters 
holdout= None 
params = {'interaction': False, 'D': 1048576, 'beta': 0.5, 'epoch': 1, 'L2': 0.5, 'L1': 0.0, 'alpha': 0.1}

# save results in lists
roc_list = []
pr_list = []

# train
learner = ftrl_proximal(**params)
learner.fit(train)
print("Model trained. Non-zero weights: ", len(learner.z) - learner.z.count(0))

# save model
learner.save("model_created_on_" + str(datetime.now()), "/mnt/nycdsa/quentin/models/")

# predict for all days
days = range(12, 23)

for day in days:
  roc_temp = 0.
  pr_temp = 0.
  print("Starting day", day)
  
  # define test
  test_temp = test[test.day == day]
  test_temp.cache()
  print("Day: ", day, "Number of impressions: ", test_temp.count())
   
  # predict
  probas_array = np.asarray(learner.predict(test_temp.drop("clicked"))) # probabilities
  clicks_array = np.asarray(test_temp.select('clicked').collect()) # labels
  # roc
  roc_temp = roc_auc_score(clicks_array, probas_array, average="macro")
  roc_list.append(roc_temp)
  print("roc_auc_score", roc_temp)
  # pr
  pr_temp = average_precision_score(clicks_array, probas_array, average="macro")
  pr_list.append(pr_temp)
  print("average_precision_score", pr_temp)

In [49]:
type(roc_list[0])
type(np.asscalar(roc_list[0]))

In [50]:
from pyspark.sql import Row

display(sqlContext.createDataFrame([Row(col1= days[i], col2=np.asscalar(roc_list[i]), col3=np.asscalar(pr_list[i])) for i in range(len(days))]))

In [51]:
from pyspark.sql import Row

display(sqlContext.createDataFrame([Row(col1= days[i], col2=np.asscalar(roc_list[i])) for i in range(len(days))]))

In [52]:
display(dbutils.fs.ls('dbfs:/mnt/nycdsa/quentin/graph'))

In [53]:
# dbutils.fs.rm("/mnt/nycdsa/quentin/models/model_created_on_2017-12-08 16:30:50.350543", recurse=True)